In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/empire_psremoting_stager.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [19]:
(
df[['@timestamp','Hostname','Channel']]

[((df['Channel'] == 'Microsoft-Windows-PowerShell/Operational') | (df['Channel'] == 'Windows PowerShell'))
    & ((df['EventID'] == 400)|(df['EventID'] == 4103))
    & ~(df['Message'].str.contains('.*Host Application%powershell.*', regex=True, na=False))
]
.head()
)

,@timestamp,Hostname,Channel
2,2020-09-20T21:08:28.401Z,WORKSTATION5.theshire.local,Microsoft-Windows-PowerShell/Operational
4,2020-09-20T21:08:28.403Z,WORKSTATION5.theshire.local,Microsoft-Windows-PowerShell/Operational
7,2020-09-20T21:08:28.406Z,WORKSTATION5.theshire.local,Microsoft-Windows-PowerShell/Operational
8,2020-09-20T21:08:28.406Z,WORKSTATION5.theshire.local,Microsoft-Windows-PowerShell/Operational
12,2020-09-20T21:08:29.400Z,WORKSTATION5.theshire.local,Microsoft-Windows-PowerShell/Operational


In [15]:
(
df[['@timestamp','Hostname','Image','Description']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (
      (df['Description'].str.lower() == 'system.management.automation')
      | (df['ImageLoaded'].str.contains('.*system.management.automation.*', regex=True))
    )
    & (~df['Image'].str.lower().str.endswith('powershell.exe', na=False))
]
)

,@timestamp,Hostname,Image,Description
768,2020-09-20T21:09:11.912Z,WORKSTATION6.theshire.local,C:\Windows\System32\wsmprovhost.exe,System.Management.Automation


In [16]:
(
df[['@timestamp','Hostname','Image','PipeName']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 17)
    & (df['PipeName'].str.lower().str.startswith('\pshost', na=False))
    & (~df['Image'].str.lower().str.endswith('powershell.exe', na=False))
]
.head()
)

,@timestamp,Hostname,Image,PipeName
799,2020-09-20T21:09:11.926Z,WORKSTATION6.theshire.local,C:\windows\system32\wsmprovhost.exe,\PSHost.132451097487065967.9036.DefaultAppDoma...
